In [3]:
import pyoxigraph as ox
import pandas as pd

In [2]:
store = ox.Store("data/wety.db")

In [3]:
def get_children(root):
    return [(w['child'].value, w['lang'].value, w['term'].value) for w in store.query(
        f"""SELECT ?child ?lang ?term WHERE {{ 
            ?sourceNode <o:item> <{root}> .
            ?child <o:source> ?sourceNode .
            ?child <o:lang> ?lang .
            ?child <o:term> ?term .
        }}"""
    )]

def get_descendants(r):
    descendants = {}
    children = get_children(r)
    for child in children:
        descendants["{}, {}".format(child[1], child[2])] = get_descendants(child[0])
    return descendants

In [6]:
root = "ǵʰelh₃-"
r = [s['w'].value for s in store.query(f'SELECT ?w WHERE {{ ?w <o:term> "{root}" }}')][0]
d = get_descendants(r)

In [11]:
data = pd.DataFrame(
    [[w['lang'].value, w['pos'].value, w['term'].value] for w in store.query(
        f"""SELECT DISTINCT ?lang ?pos ?term WHERE {{ 
            ?item <o:glossNum> ?glossNum .
            ?item <o:glossNum> ?glossNum .
            ?item <o:lang> ?lang .
            ?item <o:pos> ?pos .
            ?item <o:term> ?term .
        }}"""
    )],
    columns=["lang", "pos", "term"]
)
data.sort_values(["lang", "pos", "term"], inplace=True)
data.to_csv("data/nonzero_glossNum.csv", index=False)
print(len(data))

53331


In [12]:
data = pd.DataFrame(
    [[w['lang'].value, w['pos'].value, w['term'].value] for w in store.query(
        f"""SELECT DISTINCT ?lang ?pos ?term WHERE {{ 
            ?item <o:glossNum> ?glossNum .
            ?item <o:etyNum> ?etyNum .
            ?item <o:lang> ?lang .
            ?item <o:pos> ?pos .
            ?item <o:term> ?term .
        }}"""
    )],
    columns=["lang", "pos", "term"]
)
data.sort_values(["lang", "pos", "term"], inplace=True)
data.to_csv("data/nonzero_glossNum_and_etyNum.csv", index=False)
print(len(data))

1621
